# Hybrid Quantum Machine Learning Model @ TSpark 2022 Quantum+ Camp
by Mark Song
Futher Optimization Needed

## Import Libraries

In [ ]:
import tensorflow as tf
from tensorcircuit import keras
from tensorflow.keras import layers
import numpy as np
import deepchem as dc
import tensorcircuit as tc
import time
from functools import partial

## Tensorflow Settings

In [ ]:
tf.compat.v1.enable_eager_execution()

tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

tc.set_dtype("complex128")

## Environment Settings

In [ ]:
K = tc.set_backend("tensorflow")

## Gobal Variables

In [ ]:
n = 8
nlayers = 10
thetas = K.zeros([n,nlayers])
epochs = 50
lr = 0.001
batch_size = 64
lp = 2**n

## Generating Datasets

In [ ]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP')
train_dataset, valid_dataset, test_dataset = datasets

w_col = train_dataset.w[:, 1]
train_num = np.nonzero(w_col)
x_train=(train_dataset.X[train_num]*2-1)/32
y_train=train_dataset.y[train_num][:, 1]
w_train=train_dataset.w[train_num][:, 1]

w_col = valid_dataset.w[:,1]
val_num = np.nonzero(w_col)
x_val=(valid_dataset.X[val_num]*2-1)/32
y_val=valid_dataset.y[val_num][:, 1]
w_val=valid_dataset.w[val_num][:, 1]

w_col = test_dataset.w[:,1]
test_num = np.nonzero(w_col)
x_test=(test_dataset.X[test_num]*2-1)/32
y_test=test_dataset.y[test_num][:, 1]
w_test=test_dataset.w[test_num][:, 1]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)


test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test, w_test))
test_dataset = test_dataset.batch(batch_size)

## Creating Quantum Layer

In [ ]:
def ql_l(x, param, nl):
    c = tc.Circuit(n,inputs=x)
    for j in range(nl):
        for i in range(n):
            c.rx(i, theta=param[i, j])
        if(j%2==1):
            for i in range(n - 1):
                c.cnot(i, i + 1)
        else:
            for i in range(n - 1):
                c.cnot(n-1-i, n-2-i)
    return tc.array_to_tensor([float(K.real(c.expectation_ps(z=[i]))) for i in range(n)],dtype="float64")
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits = False)
ql = keras.QuantumLayer(partial(ql_l, nl=nlayers), [n,nlayers])

## Creating Model

In [ ]:
inputs = tf.keras.Input(shape=(1024,), name="digits")
x1 = tf.keras.layers.Dense(512, activation="relu")(inputs)
x2 = tf.keras.layers.Dropout(0.6)(x1)
x3 = tf.keras.layers.Dense(lp, activation="relu")(x2)
#x3/=K.norm(x3)
x4 = ql(x3)
x5 = tf.keras.layers.ReLU()(x4)
x6 = tf.keras.layers.Dense(1, activation="relu")(x5)
outputs = tf.keras.layers.Dense(1, activation="sigmoid", name="predictions")(x6)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.trainable
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr),
    metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()],
)

## Training Model

In [ ]:
model.fit(x=x_train,y=y_train, epochs=epochs, validation_data=val_dataset,sample_weight=w_train,batch_size=batch_size)

## Testing Model

In [ ]:
testing_result = model(x_test)
ans = 0
for i in range(len(testing_result)):
  if abs(testing_result[i]-y_test[i])<=0.5 :
    ans+=1
print(ans/len(testing_result))

auc_m = tf.keras.metrics.AUC()
auc_m.update_state(y_test, testing_result)
print(auc_m.result().numpy())